In [1]:
# Demo: Using ASDI datasets including NOAA weather data and OpenAQ air quality data to predict air quality parameters based on weather...

In [2]:
# Set up your environment according to the environment.yml file or run the following...
# %pip install boto3
# %pip install pandas
# %pip install numpy
# %pip install requests
# %pip install scikit-learn
# %pip install autogluon
# %pip install bokeh

In [3]:
import os
import boto3
import pandas as pd
import numpy as np
import sys
import requests
import json

from botocore import UNSIGNED
from botocore.config import Config
from io import StringIO

In [9]:
# Set some variables to make things a bit more dynamic. Note the following...
# 72287493134 => Downtown Los Angeles (USC), California, USA
# 72484653123 => Las Vegas-Paradise, NV, USA
# 1=pm10, 2=pm25, 7=no2, 8=co, 9=so2, 10=o3; see: https://api.openaq.org/v2/parameters (use only isCore=true); NOTE: some code assumes ID=2 for pm25!
# 7936="Los Angeles - N. Main", 1948="Compton", 8730="Pasadena", 1036="Pico Rivera", 8236="North Holywood" (data from partial ~2016/2017+)
# 1022="Las Vegas-Paradise", 344="Las Vegas-Paradise"
useLocalDataFilesIfTheyExist = True # Prevent re-fetching of data, if it's already been accessed and prepared
noaaStationId = "72484653123"
openaqParameterIds = [2]            
openaqLocationIds = ['344'] 
openaqUnhealthyThreshold = 12.0     # General EPA max allowed pm25 value in ug/m3 (pm25 is ID=2; OpenAQ measures pm25 in ug/m3)
label = 'isUnhealthy'
yearStart = 2016
yearEnd = 2022
localdatafile_weather = 'localdatafile_noaagsod.csv'
localdatafile_aq = 'localdatafile_openaq.csv'
localdatafile_merged = 'localdatafile_merged.csv'

In [20]:
# ASDI Dataset Name: NOAA GSOD
# ASDI Dataset URL : https://registry.opendata.aws/noaa-gsod/
# NOAA GSOD README : https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt
# NOAA GSOD Station Search: https://www.ncei.noaa.gov/access/search/data-search/global-summary-of-the-day
# NOAA GSOD data in S3 is organized by year and Station ID values, so this is straight-forward
# Example S3 path format => s3://noaa-gsod-pds/{yyyy}/{stationid}.csv
# Let's start with a new DataFrame and load it from a local CSV or the NOAA data source...
noaagsod_df = pd.DataFrame()

if useLocalDataFilesIfTheyExist == True and os.path.exists(localdatafile_weather):
    # Use local data file already accessed + prepared...
    print('Loading NOAA GSOD data from local file: ', localdatafile_weather)
    noaagsod_df = pd.read_csv(localdatafile_weather)
else:
    # Access + prepare data and save to a local data file...
    print('Accessing and preparing data from NOAA GSOD dataset...')
    noaagsod_bucket = 'noaa-gsod-pds'
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    for year in range(yearStart, yearEnd + 1):
        key = f'{year}/{noaaStationId}.csv'                                                          # Compute the key to get
        csv_obj = s3.get_object(Bucket=noaagsod_bucket, Key=key)                                     # Get the S3 object
        csv_string = csv_obj['Body'].read().decode('utf-8')                                          # Read object contents to a string
        noaagsod_df = pd.concat([noaagsod_df, pd.read_csv(StringIO(csv_string))], ignore_index=True) # Use the string to build the DataFrame

    # Remove unnecessary columns...
    # These are station metadata, descriptive attribute columns, features not deemed relevant, and VISIB since this is highly-correlated and cheating...
    noaagsod_df = noaagsod_df.drop(columns=['STATION', 'LATITUDE', 'LONGITUDE', 'TEMP_ATTRIBUTES', 
                                            'DEWP_ATTRIBUTES', 'MAX_ATTRIBUTES', 'MIN_ATTRIBUTES', 'PRCP_ATTRIBUTES', 
                                            'SLP_ATTRIBUTES', 'STP_ATTRIBUTES', 'VISIB_ATTRIBUTES', 'WDSP_ATTRIBUTES', 
                                            'ELEVATION', 'STP', 'SLP', 'TEMP', 'MXSPD', 'GUST', 'SNDP', 'FRSHTT', 'VISIB'])

    # Perform some Feature Engineering to append potentially useful columns to our dataset...
    # It may be true that Day of Week affects air quality (ie: higher weekday commuting/industrial pollutants; not very correlated after all)
    # It may be true that Month of Year affects air quality (ie: seasonal considerations; tends to have correlation for certain areas)
    noaagsod_df['DAYOFWEEK'] = pd.to_datetime(noaagsod_df['DATE']).dt.dayofweek + 1
    noaagsod_df['MONTH'] = pd.to_datetime(noaagsod_df['DATE']).dt.month

    # Concat/JOIN the new DAYOFWEEK column to our NOAA GSOD DataFrame AND save as CSV...
    noaagsod_df.to_csv(localdatafile_weather, index=False)

# Output noaagsod_df properties...
print('noaagsod_df.shape =', noaagsod_df.shape)
display(noaagsod_df)

Accessing and preparing data from NOAA GSOD dataset...
noaagsod_df.shape = (2342, 9)


,DATE,NAME,DEWP,WDSP,MAX,MIN,PRCP,DAYOFWEEK,MONTH
0,2016-01-01,"LAS VEGAS AIR TERMINAL, NV US",8.2,5.9,48.0,30.0,0.00,5,1
1,2016-01-02,"LAS VEGAS AIR TERMINAL, NV US",15.5,3.2,53.1,30.0,0.00,6,1
2,2016-01-03,"LAS VEGAS AIR TERMINAL, NV US",23.0,3.8,55.9,34.0,0.00,7,1
3,2016-01-04,"LAS VEGAS AIR TERMINAL, NV US",30.2,2.3,55.9,39.0,0.00,1,1
4,2016-01-05,"LAS VEGAS AIR TERMINAL, NV US",43.2,4.7,54.0,44.1,0.02,2,1
...,...,...,...,...,...,...,...,...,...
2337,2022-05-26,"LAS VEGAS AIR TERMINAL, NV US",27.9,6.8,104.0,69.1,0.00,4,5
2338,2022-05-27,"LAS VEGAS AIR TERMINAL, NV US",27.6,11.2,104.0,73.0,0.00,5,5
2339,2022-05-28,"LAS VEGAS AIR TERMINAL, NV US",29.8,9.6,102.9,79.0,0.00,6,5
2340,2022-05-29,"LAS VEGAS AIR TERMINAL, NV US",27.9,15.6,95.0,73.0,0.00,7,5


In [21]:
# ASDI Dataset Name: OpenAQ
# ASDI Dataset URL : https://registry.opendata.aws/openaq/
# OpenAQ API Docs  : https://docs.openaq.org/#/v2/
# OpenAQ S3 data is only organized by date folders, so each folder is large and contains all stations.
# Because of this, it's better to query ASDI OpenAQ data using the CloudFront-hosted API.
# Note that some days may not have values and will get filteredout via an INNER JOIN later.
# Let's start with a new DataFrame and load it from a local CSV or the NOAA data source...
aq_df = pd.DataFrame()
print('Unhealthy threshold is >', openaqUnhealthyThreshold)

if useLocalDataFilesIfTheyExist == True and os.path.exists(localdatafile_aq):
    # Use local data file already accessed + prepared...
    print('Loading OpenAQ data from local file: ', localdatafile_weather)
    aq_df = pd.read_csv(localdatafile_aq)
else:
    # Access + prepare data and save to a local data file... (NOTE: calling OpenAQ API one year at a time to avoid timeouts)
    print('Accessing and preparing data from OpenAQ dataset (HTTPS API)...')
    aq_reqUrl = "https://u50g7n0cbj.execute-api.us-east-1.amazonaws.com/v2/averages"  # OpenAQ ASDI API Endpoint URL for averages (ie: daily averages)
    for year in range(yearStart, yearEnd + 1):
        aq_reqParams = {
            'date_from': f'{year}-01-01',
            'date_to': f'{year}-12-31',
            'parameter': openaqParameterIds,
            'limit': 366,
            'page': 1,
            'offset': 0,
            'sort': 'asc',
            'spatial': 'location',
            'temporal': 'day',
            'location': openaqLocationIds,
            'group': 'true'
        }
        aq_resp = requests.get(aq_reqUrl, aq_reqParams)
        aq_data = aq_resp.json()                        # get the response data and then (below) concat to aq_df
        if(aq_data['results']):
            aq_df = pd.concat([aq_df, pd.json_normalize(aq_data['results'])], ignore_index=True)

    # Perform some Label Engineering to "bucket" our label target into {0=OKAY, 1=UNHEALTHY}
    aq_df[label] = np.where(aq_df['average'] <= openaqUnhealthyThreshold, 0, 1)

    # Using the joined dataframe, remove unnecessary columns for our final structure...
    aq_df = aq_df.drop(columns=['id', 'name', 'subtitle', 'parameterId', 'displayName', 'measurement_count'])
    
# Output aq_df properties...
print('aq_df.shape =', aq_df.shape)
display(aq_df)
aq_df.to_csv(localdatafile_aq, index=False)

Unhealthy threshold is > 12.0
Loading OpenAQ data from local file:  localdatafile_noaagsod.csv
aq_df.shape = (2130, 5)


,day,unit,average,parameter,isUnhealthy
0,2016-03-06,µg/m³,2.8500,pm25,0
1,2016-03-07,µg/m³,5.1500,pm25,0
2,2016-03-10,µg/m³,14.8500,pm25,1
3,2016-03-11,µg/m³,2.9143,pm25,0
4,2016-03-12,µg/m³,8.5773,pm25,0
...,...,...,...,...,...
2125,2022-06-02,µg/m³,6.0708,pm25,0
2126,2022-06-03,µg/m³,5.1000,pm25,0
2127,2022-06-04,µg/m³,4.8708,pm25,0
2128,2022-06-05,µg/m³,5.2625,pm25,0


In [22]:
# Merge the NOAA GSOD weather data with our OpenAQ data by DATE...
# Perform another column drop to remove columns we don't want as features/inputs.
merged_df = pd.merge(noaagsod_df, aq_df, how="inner", left_on="DATE", right_on="day")
merged_df = merged_df.drop(columns=['DATE', 'NAME', 'day', 'unit', 'average', 'parameter'])
print('merged_df.shape =', merged_df.shape)
display(merged_df)
merged_df.to_csv(localdatafile_merged, index=False)

# TODO: Explore other ways to join the data and handle null values.

merged_df.shape = (2123, 8)


,DEWP,WDSP,MAX,MIN,PRCP,DAYOFWEEK,MONTH,isUnhealthy
0,37.4,15.9,69.1,55.0,0.00,7,3,0
1,30.4,5.0,69.1,42.1,0.01,1,3,0
2,24.2,4.1,79.0,51.1,0.00,4,3,1
3,23.2,7.2,79.0,51.1,0.00,5,3,0
4,31.8,10.9,79.0,46.9,0.00,6,3,0
...,...,...,...,...,...,...,...,...
2118,27.9,6.8,104.0,69.1,0.00,4,5,0
2119,27.6,11.2,104.0,73.0,0.00,5,5,0
2120,29.8,9.6,102.9,79.0,0.00,6,5,0
2121,27.9,15.6,95.0,73.0,0.00,7,5,1


In [23]:
# Additional import statements and specify target label column and split our data...
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
eval_metric='accuracy'
df_train,df_test = train_test_split(merged_df, test_size=0.33, random_state=1)
df_train.shape, df_test.shape
print('Number of training samples:', len(df_train))
print('Number of test samples:', len(df_test))

Number of training samples: 1422
Number of test samples: 701


In [24]:
#Get test data and remove the target label column...
test_data=df_test.drop([label],axis=1)
display(test_data)

,DEWP,WDSP,MAX,MIN,PRCP,DAYOFWEEK,MONTH
1665,20.6,3.2,62.1,35.1,0.00,4,12
1201,43.6,4.4,107.1,80.1,0.00,2,8
1381,19.6,15.2,73.0,45.0,0.06,2,3
1655,6.3,2.9,64.0,34.0,0.00,7,12
1059,36.2,10.1,78.1,57.0,0.00,6,4
...,...,...,...,...,...,...,...
1794,24.1,4.4,98.1,64.0,0.00,5,4
211,24.3,8.2,91.9,62.1,0.00,5,10
1276,25.8,1.5,75.0,50.0,0.00,5,11
982,33.4,3.0,61.0,42.1,0.00,6,1


In [25]:
# Use AutoGluon TabularPredictor to fit a model for our training data...
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path='AutogluonModels/aq_by_weather/').fit(train_data=df_train, verbosity=2, presets='best_quality')

Presets specified: ['best_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/aq_by_weather/"
AutoGluon Version:  0.4.0
Python Version:     3.9.12
Operating System:   Linux
Train Data Rows:    1422
Train Data Columns: 7
Label Column: isUnhealthy
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    14932.67 MB
	Train Data (Original)  Memory Usage: 0.08 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadat

In [26]:
# View AutoGluon model leaderboard...
predictor.leaderboard(df_train, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,0.810127,0.020698,0.020924,0.004591,0.020698,0.020924,0.004591,1,True,2
1,RandomForestEntr_BAG_L1,1.000000,0.826301,0.134501,0.200320,1.038539,0.134501,0.200320,1.038539,1,True,6
2,RandomForestGini_BAG_L1,1.000000,0.825598,0.142367,0.190549,0.829067,0.142367,0.190549,0.829067,1,True,5
3,ExtraTreesEntr_BAG_L1,1.000000,0.832630,0.393932,0.156275,0.746335,0.393932,0.156275,0.746335,1,True,9
4,ExtraTreesGini_BAG_L1,1.000000,0.828411,0.504236,0.154830,0.748714,0.504236,0.154830,0.748714,1,True,8
5,LightGBMLarge_BAG_L1,0.998594,0.830520,0.192609,0.051669,9.057532,0.192609,0.051669,9.057532,1,True,13
6,XGBoost_BAG_L1,0.947961,0.837553,0.072032,0.065264,4.132747,0.072032,0.065264,4.132747,1,True,11
7,NeuralNetTorch_BAG_L2,0.935302,0.847398,2.744180,1.483891,69.702571,0.335515,0.248348,10.330227,2,True,24
8,LightGBM_BAG_L1,0.893812,0.847398,0.063096,0.029077,6.377447,0.063096,0.029077,6.377447,1,True,4
9,LightGBMXT_BAG_L2,0.889592,0.854430,2.470488,1.269317,65.785712,0.061822,0.033775,6.413368,2,True,15


In [27]:
# View feature importance... (TODO: ADD VISUALIZATION)
predictor.feature_importance(data=df_train)

Computing feature importance via permutation shuffling for 7 features using 1000 rows with 3 shuffle sets...
	57.71s	= Expected runtime (19.24s per shuffle set)
	18.88s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
WDSP,0.068667,0.004163,0.000612,3,0.092523,0.044810
MONTH,0.068333,0.008083,0.002316,3,0.114649,0.022017
MIN,0.049333,0.004933,0.001658,3,0.077599,0.021067
MAX,0.026667,0.002309,0.001245,3,0.039900,0.013434
DEWP,0.010667,0.005508,0.039270,3,0.042226,-0.020892
PRCP,0.002333,0.001155,0.036414,3,0.008950,-0.004283
DAYOFWEEK,0.001667,0.003055,0.222222,3,0.019172,-0.015839


In [28]:
# Making Predictions...
y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred,columns=[label])
display(y_pred)

,isUnhealthy
1665,1
1201,0
1381,0
1655,1
1059,0
...,...
1794,0
211,0
1276,1
982,1


In [29]:
# Evaluate Model Performance...
predictor.evaluate(df_test)

Evaluation: accuracy on test data: 0.8445078459343794
Evaluations on test data:
{
    "accuracy": 0.8445078459343794,
    "balanced_accuracy": 0.7616114212863234,
    "mcc": 0.5587192531848527,
    "roc_auc": 0.8511561964215062,
    "f1": 0.653968253968254,
    "precision": 0.7202797202797203,
    "recall": 0.5988372093023255
}


{'accuracy': 0.8445078459343794,
 'balanced_accuracy': 0.7616114212863234,
 'mcc': 0.5587192531848527,
 'roc_auc': 0.8511561964215062,
 'f1': 0.653968253968254,
 'precision': 0.7202797202797203,
 'recall': 0.5988372093023255}

In [30]:
# Display final results (TODO: ADD VISUALIZATION)
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.862166       1.314244  79.010305                0.002784           0.894513            3       True         26
1           LightGBM_BAG_L2   0.861463       1.269513  66.856566                0.033971           7.484222            2       True         16
2           CatBoost_BAG_L2   0.859353       1.277490  70.631570                0.041947          11.259226            2       True         19
3            XGBoost_BAG_L2   0.857947       1.304247  65.423222                0.068704           6.050878            2       True         23
4      LightGBMLarge_BAG_L2   0.855837       1.284796  71.587299                0.049254          12.214955            2       True         25
5       WeightedEnsemble_L2   0.855134       0.280903  13.299856                

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
